In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Untitled1.ipynb",
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "metadata": {
        "id": "bbYK2QkXJ5Pj"
      },
      "source": [
        "!pip install boto3"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "HlkZhcRdbsO0"
      },
      "source": [
        "AWS_KEY_ID = '*******'\n",
        "AWS_SECRET_KEY = '**********'\n",
        "\n",
        "IMAGE_PATH_ON_S3 = 'dataset_v1_ratio_0.5/content/xmls/1530614630188935288dbe5dd10f2b14ddca3e11b7aeb20d7a5.png'\n",
        "IMAGE_LOCAL_PATH = '/content/here/hey.png'\n",
        "S3_BUCKET_NAME = 'airborne-object-detection'"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "zbrN1i6NQScK"
      },
      "source": [
        "import boto3\n",
        "import botocore\n",
        "import os"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "PfrXI4mBZvMn"
      },
      "source": [
        "s3 = boto3.client('s3', aws_access_key_id=AWS_KEY_ID, aws_secret_access_key=AWS_SECRET_KEY)"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "_rWHFXwlb341"
      },
      "source": [
        "Downloading only one file from s3 bucket"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "lQ8OV5agZyt8"
      },
      "source": [
        "def download_file(bucket_name=S3_BUCKET_NAME, file_local= IMAGE_LOCAL_PATH, s3_path= IMAGE_PATH_ON_S3, client=s3):\n",
        "    \"\"\"\n",
        "    params:\n",
        "    - bucket_name: s3 bucket name\n",
        "    - file_local: local path to folder in which to place files along with the file type\n",
        "    - s3_path: full file path that needs to be download\n",
        "    - client: initialized s3 client object\n",
        "    \"\"\"\n",
        "    client.download_file( bucket_name, s3_path, file_local)"
      ],
      "execution_count": 34,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "NYxri_bwcB4G"
      },
      "source": [
        "download_file()"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "GC06BVuwb8Md"
      },
      "source": [
        "downloading a directory from s3 bucket"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "b4LxuxzOXzYe"
      },
      "source": [
        "def download_dir(prefix, local, bucket, client=s3):\n",
        "    \"\"\"\n",
        "    params:\n",
        "    - prefix: pattern to match in s3\n",
        "    - local: local path to folder in which to place files\n",
        "    - bucket: s3 bucket with target contents\n",
        "    - client: initialized s3 client object\n",
        "    \"\"\"\n",
        "    keys = []\n",
        "    dirs = []\n",
        "    next_token = ''\n",
        "    base_kwargs = {\n",
        "        'Bucket':bucket,\n",
        "        'Prefix':prefix,\n",
        "    }\n",
        "    while next_token is not None:\n",
        "        kwargs = base_kwargs.copy()\n",
        "        if next_token != '':\n",
        "            kwargs.update({'ContinuationToken': next_token})\n",
        "        results = client.list_objects_v2(**kwargs)\n",
        "        contents = results.get('Contents')\n",
        "        for i in contents:\n",
        "            k = i.get('Key')\n",
        "            if k[-1] != '/':\n",
        "                keys.append(k)\n",
        "            else:\n",
        "                dirs.append(k)\n",
        "        next_token = results.get('NextContinuationToken')\n",
        "    for d in dirs:\n",
        "        dest_pathname = os.path.join(local, d)\n",
        "        if not os.path.exists(os.path.dirname(dest_pathname)):\n",
        "            os.makedirs(os.path.dirname(dest_pathname))\n",
        "    for k in keys:\n",
        "        dest_pathname = os.path.join(local, k)\n",
        "        if not os.path.exists(os.path.dirname(dest_pathname)):\n",
        "            os.makedirs(os.path.dirname(dest_pathname))\n",
        "        client.download_file(bucket, k, dest_pathname)"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "CuRTV9IsYGI7"
      },
      "source": [
        "download_dir(\"dataset_v1_ratio_0.5/\", '/content/here', 'airborne-object-detection', client=s3)"
      ],
      "execution_count": null,
      "outputs": []
    }
  ]
}